In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP

%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects_ser.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [3]:
#make symbolic links and directory structure that fix expects
for sub in sub_list:
    for exp in ['ser','sim']:
        for run in ['1','2','3']:

            sub_dir = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + run + '/'
            
            if os.path.exists(sub_dir):
                melodic_dir = sub_dir + 'melodic/'
                ica_outputs_dir = melodic_dir + 'filtered_func_data.ica/'
                reg_dir = melodic_dir + 'reg/'
                motion_dir = melodic_dir + 'mc/'

                if not os.path.exists(ica_outputs_dir):
                    os.mkdir(ica_outputs_dir)         

                #move ica outputs from melodic dir to ica outputs dur
                cmd_str = 'mv ' + melodic_dir + '* ' + ica_outputs_dir
                os.system(cmd_str)

                if not os.path.exists(reg_dir):
                    os.mkdir(reg_dir)
                if not os.path.exists(motion_dir):
                    os.mkdir(motion_dir)

                old = sub_dir + 'unsmoothed_timeseries.nii.gz'
                new = melodic_dir + 'filtered_func_data.nii.gz'
                cmd_str = 'ln -s ' + old + ' ' + new
                if not os.path.exists(new):
                    os.system(cmd_str)

                old = sub_dir + 'functional_mask.nii.gz'
                new = melodic_dir + 'mask.nii.gz'
                cmd_str = 'ln -s ' + old + ' ' + new
                if not os.path.exists(new):
                    os.system(cmd_str)

                old = sub_dir + 'mean_func.nii.gz'
                new = melodic_dir + 'mean_func.nii.gz'
                cmd_str = 'ln -s ' + old + ' ' + new
                if not os.path.exists(new):
                    os.system(cmd_str)

                old = sub_dir + 'example_func.nii.gz'
                new = reg_dir + 'example_func.nii.gz'
                cmd_str = 'ln -s ' + old + ' ' + new
                if not os.path.exists(new):
                    os.system(cmd_str)

                old = sub_dir + 'func2anat_flirt.mat'
                new = reg_dir + 'highres2example_func.mat'
                cmd_str = 'convert_xfm -omat ' + new + ' -inverse ' + old
                if not os.path.exists(new):
                    os.system(cmd_str)

                old = home_dir + 'analysis/workingdir/antswarp/_subject_id_' + sub + '/convertbrain/brain_out.nii.gz'
                new = reg_dir + 'highres.nii.gz'
                cmd_str = 'ln -s ' + old + ' ' + new
                if not os.path.exists(new):
                    os.system(cmd_str)

                ##deal with motion data
                data = pd.DataFrame.from_csv(sub_dir + 'realignment_params.csv')
                data.to_csv(motion_dir + 'prefiltered_func_data_mcf.par',' ',
                            columns = ['rot_x','rot_y','rot_z','trans_x','trans_y','trans_z'],
                           header = False, index = False)
            else:
                print exp + ' ' + sub + ' ' + run


ser fd_127 3
sim fd_127 3
sim fd_133 3


In [4]:
def extract_features(in_tuple):
    thresholds = ['5','10','15','20','25','30']
    sub = in_tuple[0]
    exp = in_tuple[1]
    run = str(in_tuple[2])
    
    fix_cmd = '/home/iballard/fix1.06/fix'
    sub_path = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + run + '/melodic'
    training_file = '/home/iballard/fix1.06/training_files/WhII_MB6.RData'

    if os.path.exists(sub_path):
        cmd_str = fix_cmd + ' -f ' + sub_path
        os.system(cmd_str)
        
        for thresh in thresholds:
            cmd_str = fix_cmd + ' -c ' + sub_path + ' ' + training_file + ' ' + thresh
            os.system(cmd_str)

In [ ]:
in_tuples = []
for sub in sub_list:
    for exp in ['ser','sim']:
        for run in range(1,4):
            in_tuples.append((sub,exp,run))

pool = multiprocessing.Pool(processes = 24)
pool.map(extract_features,in_tuples)